In [2]:
!pip install transformers datasets evaluate rouge_score accelerate


In [3]:
import transformers

transformers.__version__

'4.44.2'

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from huggingface_hub import list_datasets

datasets_list = list_datasets()

num = 0
for dataset in datasets_list:
  print(dataset)

  if (num > 10):
    break

  num += 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetInfo(id='amirveyseh/acronym_identification', author='amirveyseh', sha='15ef643450d589d5883e289ffadeb03563e80a9e', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 39, 57, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=925, downloads_all_time=None, likes=19, paperswithcode_id='acronym-identification', tags=['task_categories:token-classification', 'annotations_creators:expert-generated', 'language_creators:found', 'multilinguality:monolingual', 'source_datasets:original', 'language:en', 'license:mit', 'size_categories:10K<n<100K', 'format:parquet', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'arxiv:2010.14678', 'region:us', 'acronym-identification'], card_data=None, siblings=None)
DatasetInfo(id='ade-benchmark-corpus/ade_corpus_v2', author='ade-benchmark-corpus', sha='4ba01c71687dd7c996597042449448ea312126cf', 

In [7]:
from datasets import load_dataset

cnn_news_summary_ds = load_dataset("fka/awesome-chatgpt-prompts")

cnn_news_summary_ds

prompts.csv:   0%|          | 0.00/84.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/170 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['act', 'prompt'],
        num_rows: 170
    })
})

In [8]:
cnn_news_summary_ds.shape

{'train': (170, 2)}

In [11]:
print(cnn_news_summary_ds["train"].info.description)

In [13]:
cnn_news_summary_ds["train"].features

{'act': Value(dtype='string', id=None),
 'prompt': Value(dtype='string', id=None)}

In [15]:
print(cnn_news_summary_ds["train"].info.homepage)

In [17]:
cnn_news_summary_ds["train"][0]

{'act': 'An Ethereum Developer',
 'prompt': 'Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.'}

In [19]:
from datasets import DatasetDict

# Assuming cnn_news_summary_ds is your original DatasetDict

# Split the 'train' dataset into train and test
train_test_ds = cnn_news_summary_ds["train"].train_test_split(test_size=0.2, seed=42)

# Update the original DatasetDict with the new splits
cnn_news_summary_ds = DatasetDict({
    "train": train_test_ds["train"],
    "test": train_test_ds["test"],
    # If you have a validation set, keep it:
    # "validation": cnn_news_summary_ds["validation"]
})

cnn_news_summary_ds

DatasetDict({
    train: Dataset({
        features: ['act', 'prompt'],
        num_rows: 136
    })
    test: Dataset({
        features: ['act', 'prompt'],
        num_rows: 34
    })
})

In [22]:
def clean_txt(example):
    for txt in ["article", "highlights"]:
       example[txt]  = example[txt].lower()
       example[txt]  = example[txt].replace("\\", "")
       example[txt]  = example[txt].replace("/", "")
       example[txt]  = example[txt].replace("\n", "")
       example[txt]  = example[txt].replace("``", "")
       example[txt]  = example[txt].replace('"', '')
       example[txt]  = example[txt].replace("--", "") # Replacing double hyphens

    return example

In [23]:
cleaned_cnn_news_summary_ds = cnn_news_summary_ds.map(clean_txt)

cleaned_cnn_news_summary_ds

Map:   0%|          | 0/136 [00:00<?, ? examples/s]

KeyError: 'article'